In [162]:
from cryptography.hazmat.primitives.ciphers import algorithms, modes, Cipher
import common

In [163]:

# The library seems to lack a raw AES function. Instead, we will use a Cipher object in
# ECB mode to do AES decryption on each block, and then manually handle the details of
# CBC mode ourselves.

def aes_decrypt(key, block):
    decryptor = Cipher(
        algorithms.AES128(key),
        modes.ECB(),
    ).decryptor()
    return decryptor.update(block) + decryptor.finalize()



def aes_encrypt(key, block):
    encryptor = Cipher(
        algorithms.AES128(key),
        modes.ECB(),
    ).encryptor()
    return encryptor.update(block) + encryptor.finalize()

In [164]:
def decrypt_aes_cbc_block(ctext_prev, ctext, key):
    return bytes([b1 ^ b2 for b1, b2 in zip(aes_decrypt(key, ctext), ctext_prev)])
def decrypt_aes_cbc(ciphertext, key):
    blocks_w_padding = [decrypt_aes_cbc_block(ciphertext[i:i+16], ciphertext[i+16:i+32], key)
     for i in range(0, len(ciphertext) - 32 + 1, 16)]
    # the last character of the last block will tell us how much padding there is.
    padding = int(blocks_w_padding[-1][-1])
    print("Padding: ", padding)
    # Concatenate 
    return bytes([
        byte for bytes in blocks_w_padding[:-1] + [blocks_w_padding[-1][:-padding]]
        for byte in bytes])

In [165]:
def decrypt_aes_ctr_block(block, block_num, key, iv):
    pad = aes_encrypt(key, ((int.from_bytes(iv, "big") + block_num) % (1 << 128)).to_bytes(16, "big"))
    return bytes([ctext ^ p for ctext, p in zip(block, pad)])
    
def decrypt_aes_ctr(ciphertext, key):
    # Peel off the IV from the beginning of the ciphertext.
    iv, ciphertext = ciphertext[:16], ciphertext[16:]
    # Split the ciphertext into 16-byte blocks.
    blocks_enc = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    return bytes([byte for block, block_num in zip(blocks_enc, range(len(blocks_enc))) for byte in decrypt_aes_ctr_block(block, block_num, key, iv)])


## Problem 1

In [166]:
key = bytes.fromhex("140b41b22a29beb4061bda66b6747e14")
ciphertext = bytes.fromhex("4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81")
decrypt_aes_cbc(ciphertext, key)

Padding:  8


b'Basic CBC mode encryption needs padding.'

In [167]:
# Check our work using the normal end-to-end decryption.

decryptor = Cipher(
        algorithms.AES128(key),
        modes.CBC(ciphertext[:16])
    ).decryptor()
decryptor.update(ciphertext[16:]) + decryptor.finalize()

b'Basic CBC mode encryption needs padding.\x08\x08\x08\x08\x08\x08\x08\x08'

## Problem 2

In [171]:
key = bytes.fromhex("140b41b22a29beb4061bda66b6747e14")
ciphertext = bytes.fromhex("5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253")
decrypt_aes_cbc(ciphertext, key)

Padding:  16


b'Our implementation uses rand. IV'

In [172]:
# Check our work using the normal end-to-end decryption.

decryptor = Cipher(
        algorithms.AES128(key),
        modes.CBC(ciphertext[:16])
    ).decryptor()
decryptor.update(ciphertext[16:]) + decryptor.finalize()

b'Our implementation uses rand. IV\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'

## Problem 3

In [169]:
key = bytes.fromhex("36f18357be4dbd77f050515c73fcf9f2")
ciphertext = bytes.fromhex("69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329")
decrypt_aes_ctr(ciphertext, key)

b'CTR mode lets you build a stream cipher from a block cipher.'

In [170]:
key = bytes.fromhex("36f18357be4dbd77f050515c73fcf9f2")
ciphertext = bytes.fromhex("770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451")
decrypt_aes_ctr(ciphertext, key)

b'Always avoid the two time pad!'